In [1]:
import pandas as pd
import numpy as np
import math
import joblib
import dill as pickle
import requests
import json
import warnings

from matplotlib import pyplot as plt

import seaborn as sns

from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import mean_absolute_error, classification_report, confusion_matrix, accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sqlalchemy import create_engine

from tqdm import tqdm

from MarkovChain import MarkovChain

sns.set()

pd.options.display.float_format = '{:.4f}'.format

In [2]:
# параметры подключения
username = '%user%'
password = '%password%'
host = '%host%'
port = '%port%'
database = '%database%'

# Создание строки подключения
connection_string = f'postgresql://{username}:{password}@{host}:{port}/{database}'

# Создание подключения к PostgreSQL с использованием sqlalchemy
engine = create_engine(connection_string)

# Загрузка таблицы в DataFrame
table_name = 'ds_gavrilova'  # Укажите имя таблицы
df = pd.read_sql_table(table_name, engine)

# Вывод DataFrame на экран
df.head(10)


,user_id,cur_date,time_zone,age,tg_bot,payment,course_id,day_from_start,day_to_finish,auth_date_last_delay,...,view_optional_rate,view_attestation_rate,activity_views_mean,activity_required_views_mean,activity_optional_views_mean,activity_attestation_views_mean,activity_view_date_last_delay,activity_views_interval,activity_view_shown_delay_first_mean,target
0,982,2023-12-01 21:00:00+00:00,3.0000,16.0000,0,0,3,1.0000,60.0000,1.1050,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,1.0000,1.0000,0
1,982,2023-12-02 21:00:00+00:00,3.0000,16.0000,0,0,3,2.0000,59.0000,2.1050,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2.0000,2.0000,2.0000,0
2,982,2023-12-03 21:00:00+00:00,3.0000,16.0000,0,0,3,3.0000,58.0000,3.1050,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,3.0000,3.0000,3.0000,0
3,982,2023-12-04 21:00:00+00:00,3.0000,16.0000,0,0,3,4.0000,57.0000,4.1050,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,4.0000,4.0000,4.0000,0
4,982,2023-12-05 21:00:00+00:00,3.0000,16.0000,0,0,3,5.0000,56.0000,5.1050,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,5.0000,5.0000,5.0000,0
5,982,2023-12-06 21:00:00+00:00,3.0000,16.0000,0,0,3,6.0000,55.0000,0.3481,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,6.0000,6.0000,6.0000,0
6,982,2023-12-07 21:00:00+00:00,3.0000,16.0000,0,0,3,7.0000,54.0000,1.3481,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,7.0000,7.0000,7.0000,0
7,982,2023-12-08 21:00:00+00:00,3.0000,16.0000,0,0,3,8.0000,53.0000,2.3481,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,8.0000,8.0000,8.0000,0
8,982,2023-12-09 21:00:00+00:00,3.0000,16.0000,0,0,3,9.0000,52.0000,3.3481,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,9.0000,9.0000,9.0000,0
9,982,2023-12-10 21:00:00+00:00,3.0000,16.0000,0,0,3,10.0000,51.0000,4.3481,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,10.0000,10.0000,10.0000,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369131 entries, 0 to 369130
Data columns (total 44 columns):
 #   Column                                       Non-Null Count   Dtype              
---  ------                                       --------------   -----              
 0   user_id                                      369131 non-null  int64              
 1   cur_date                                     369131 non-null  datetime64[ns, UTC]
 2   time_zone                                    359005 non-null  float64            
 3   age                                          316095 non-null  float64            
 4   tg_bot                                       369131 non-null  int64              
 5   payment                                      369131 non-null  int64              
 6   course_id                                    369131 non-null  int64              
 7   day_from_start                               369131 non-null  float64            
 8   day_to_finish 

# Построение модели определения вероятности завершения курса

In [4]:
# отделяем признаки от целевой переменной
features = df.drop(['user_id', 'cur_date', 'target'], axis=1)
target = df['target']

In [5]:
# разделение данных на обучающую и тестовую выборки
X_train, X_val, y_train, y_val = train_test_split(features, target, test_size=0.4, random_state=12)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.2, random_state=12)

In [6]:
# класс для работы с выбросами
class GetRidOfEmissions(TransformerMixin, BaseEstimator):

    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        for col in ['view_rate', 'view_required_rate', 'view_optional_rate', 'view_attestation_rate', 'progress',
            'exercise_rate', 'exercise_required_rate', 'exercise_optional_rate', 'exercise_attestation_rate']:
            X[col] = X[col].apply(lambda x: x if x <= 100 else 100)
        return X

In [7]:
# конвейер
pipe = Pipeline([ 
    ('emissions', GetRidOfEmissions()), # работа с выросами
    ('imputer', SimpleImputer(strategy='median')),  # заполнение пропусков
    ('scaler', StandardScaler()),  # нормализация признаков
    ('classify', RandomForestClassifier(class_weight='balanced', random_state=12))
])

In [8]:
pipe.fit(X=X_train, y=y_train)
print('Качество модели на обучающей выборке:', {roc_auc_score(y_train, pipe.predict_proba(X_train)[:, 1])})

print('Качество модели на валидационной выборке:', {roc_auc_score(y_val, pipe.predict_proba(X_val)[:, 1])})

print('Качество модели на тестовой выборке:', {roc_auc_score(y_test, pipe.predict_proba(X_test)[:, 1])})

Качество модели на обучающей выборке: {0.9999913429039015}
Качество модели на валидационной выборке: {0.997656591610537}
Качество модели на тестовой выборке: {0.997591756333008}


In [9]:
# сериализация модели
with open('models/successful_attestation_probability_model.pk', 'wb') as file:
    pickle.dump(pipe, file)

In [10]:
# проверка правильности загрузки модели
with open('models/successful_attestation_probability_model.pk','rb') as f:
    loaded_model = pickle.load(f)
    
print(
    'Качество модели на тестовой выборке от законсервированной модели:', 
    {roc_auc_score(y_test, loaded_model.predict_proba(X_test)[:, 1])}
)

Качество модели на тестовой выборке от законсервированной модели: {0.997591756333008}


# Определение количества классов учащихся и распределения значений основных метрик согласно определенным классам

In [11]:
# предобработка
# работа с выбросами
for col in ['view_rate', 'view_required_rate', 'view_optional_rate', 'view_attestation_rate', 'progress',
            'exercise_rate', 'exercise_required_rate', 'exercise_optional_rate', 'exercise_attestation_rate']:
    features[col] = features[col].apply(lambda x: x if x <= 100 else 100)

# заполнение пропусков
features.fillna({'age': features['age'].describe()['50%']}, inplace=True)
features.fillna({'time_zone': features['time_zone'].value_counts().idxmax()}, inplace=True)

In [12]:
# масштабирование признаков
scaler = StandardScaler()
features_norm = scaler.fit_transform(features.values)

In [13]:
# определение кластеров
model = KMeans(3, random_state=12)
clusters = model.fit_predict(features_norm)

print("Центроиды кластеров:")
print(model.cluster_centers_)

Центроиды кластеров:
[[-0.07573727  0.02180655 -0.03899764  0.2325713  -0.03677565 -0.58713397
   0.54014386 -0.1504736  -0.128808   -0.16478971  0.00533942 -0.23654396
  -0.20185673 -0.1797392  -0.01259163 -0.23342144 -0.19219676 -0.56180317
  -0.5677487  -0.40451402 -0.47998638  0.19440429  0.25580122 -0.11918106
  -0.08877137 -0.53790178 -0.63028281 -0.62718716 -0.51348677 -0.01742745
  -0.1824813  -0.24420529 -0.17211975 -0.23766556 -0.21283377 -0.23920261
  -0.20247376 -0.12444644 -0.32142508 -0.556969   -0.46765415]
 [-0.21805294  0.01647911  0.18485747 -0.09403056  0.47993472  0.92594497
  -0.32187159 -0.65272603 -0.53336713  2.07702663  1.64913775  1.78578156
   1.38576924  2.09463942  1.70506234  1.77692211  1.32205882 -0.9091508
  -0.85400036 -0.70490824  0.07843289  1.12836546  0.75739193  1.29330316
   0.61823583 -0.85204008 -0.31996605 -0.33986824 -0.90426763  1.79133137
   1.97480587  2.22024989  1.91915949  1.63255424  1.8629772   1.90391584
   1.8051996   0.98528291 -1.

In [14]:
df['cluster'] = clusters.tolist()

In [15]:
df_clusters = df.pivot_table(values=['cluster'], index=['user_id'], columns=['cur_date']).sample(20)
df_clusters

cluster                            \
cur_date 2023-11-30 21:00:00+00:00 2023-12-01 21:00:00+00:00   
user_id                                                        
42951                       0.0000                    0.0000   
76636                       0.0000                    0.0000   
35866                          NaN                    0.0000   
54367                          NaN                    0.0000   
77283                          NaN                    0.0000   
68469                          NaN                    0.0000   
72761                          NaN                    0.0000   
53196                       0.0000                    0.0000   
67798                       0.0000                    0.0000   
68163                          NaN                    0.0000   
42128                          NaN                    0.0000   
72482                          NaN                    0.0000   
66764                       0.0000                    0.0000   
73427                       0.0000                    0.0000   
40881                          NaN                    0.0000   
53543                          NaN                    0.0000   
74258                       0.0000                    0.0000   
4637                           NaN                       NaN   
5920                           NaN                       NaN   
72728                          NaN                    0.0000   

                                                              \
cur_date 2023-12-02 21:00:00+00:00 2023-12-03 21:00:00+00:00   
user_id                                                        
42951                       0.0000                    0.0000   
76636                       0.0000                    0.0000   
35866                       0.0000                    0.0000   
54367                       0.0000                    0.0000   
77283                       0.0000                    0.0000   
68469                       0.0000                    0.0000   
72761                       0.0000                    0.0000   
53196                       0.0000                    0.0000   
67798                       0.0000                    0.0000   
68163                       0.0000                    0.0000   
42128                       0.0000                    0.0000   
72482                       0.0000                    0.0000   
66764                       0.0000                    0.0000   
73427                       0.0000                    0.0000   
40881                       0.0000                    0.0000   
53543                       0.0000                    0.0000   
74258                       0.0000                    0.0000   
4637                           NaN                       NaN   
5920                           NaN                       NaN   
72728                       0.0000                    0.0000   

                                                              \
cur_date 2023-12-04 21:00:00+00:00 2023-12-05 21:00:00+00:00   
user_id                                                        
42951                       0.0000                    0.0000   
76636                       0.0000                    0.0000   
35866                       0.0000                    0.0000   
54367                       0.0000                    0.0000   
77283                       0.0000                    0.0000   
68469                       0.0000                    0.0000   
72761                       0.0000                    0.0000   
53196                       0.0000                    0.0000   
67798                       0.0000                    0.0000   
68163                       0.0000                    0.0000   
42128                       0.0000                    0.0000   
72482                       0.0000                    0.0000   
66764                       0.0000                    0.0000   
73427                       0.0000                    0.0

In [16]:
metrics_descr_0 = df[df['cluster'] == 0][features.columns].describe().T
metrics_descr_1 = df[df['cluster'] == 1][features.columns].describe().T
metrics_descr_2 = df[df['cluster'] == 2][features.columns].describe().T

clusters_metrics_descr = pd.concat([metrics_descr_0, metrics_descr_1, metrics_descr_2], axis=1)
clusters_metrics_descr.head(60)

# clusters_metrics_descr.to_excel('2024_11_12_clusters_metrics.xlsx')

,count,mean,std,min,25%,50%,75%,max,count,mean,...,75%,max,count,mean,std,min,25%,50%,75%,max
time_zone,193810.0000,3.6245,1.4898,-8.0000,3.0000,3.0000,3.0000,12.0000,46832.0000,3.3823,...,3.0000,12.0000,118363.0000,4.1043,1.8131,2.0000,3.0000,3.0000,5.0000,12.0000
age,176951.0000,17.4801,2.4368,13.0000,16.0000,17.0000,18.0000,61.0000,20244.0000,17.9651,...,18.0000,59.0000,118900.0000,17.2886,2.3012,13.0000,16.0000,17.0000,18.0000,61.0000
tg_bot,198708.0000,0.5775,0.6992,0.0000,0.0000,0.0000,1.0000,2.0000,47014.0000,0.7352,...,1.0000,2.0000,123409.0000,0.5995,0.7648,0.0000,0.0000,0.0000,1.0000,2.0000
payment,198708.0000,0.4010,0.4901,0.0000,0.0000,0.0000,1.0000,1.0000,47014.0000,0.2521,...,1.0000,1.0000,123409.0000,0.1405,0.3475,0.0000,0.0000,0.0000,0.0000,1.0000
course_id,198708.0000,50.7801,25.1736,3.0000,49.0000,49.0000,71.0000,83.0000,47014.0000,64.8178,...,82.0000,83.0000,123409.0000,48.4145,29.9371,3.0000,3.0000,71.0000,71.0000,83.0000
day_from_start,198708.0000,21.4669,15.7607,1.0000,9.0000,18.0000,30.0000,90.0000,47014.0000,51.0703,...,61.0000,90.0000,123409.0000,44.5314,11.3456,11.0000,35.0000,44.0000,53.0000,90.0000
day_to_finish,198708.0000,42.5126,16.5768,0.0000,34.0000,44.0000,53.0000,89.0000,47014.0000,25.6440,...,41.0000,85.0000,123409.0000,17.3397,10.6522,0.0000,8.0000,17.0000,26.0000,64.0000
auth_date_last_delay,198708.0000,77.0576,124.3035,0.0002,8.1642,26.6402,58.1782,474.1144,47014.0000,6.4701,...,4.1682,480.1144,123409.0000,167.2576,158.6386,0.0022,49.3328,86.0539,388.9585,480.1144
auth_interval,198708.0000,63.5282,128.8786,0.0053,1.8905,5.5215,24.2390,474.1144,47014.0000,4.2978,...,2.0484,480.1144,123409.0000,142.5485,174.7673,0.0288,8.2507,31.0770,388.9585,480.1144
exercise_rate,198708.0000,5.8478,7.6374,0.0000,0.0000,0.0000,11.6667,64.7059,47014.0000,41.5487,...,55.7377,266.6667,123409.0000,0.0891,0.7920,0.0000,0.0000,0.0000,0.0000,32.9897


# Построение модели определения класса учащегося

In [17]:
# определение целевой переменной
target = df['cluster'].astype('category')

In [18]:
# разделение данных на обучающую и тестовую выборки
X_train, X_val, y_train, y_val = train_test_split(features, target, test_size=0.4, random_state=12)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.2, random_state=12)

In [19]:
# конвейер
pipe_cluster = Pipeline([ 
    ('emissions', GetRidOfEmissions()), # работа с выросами
    ('imputer', SimpleImputer(strategy='median')),  # заполнение пропусков
    ('scaler', StandardScaler()),  # нормализация признаков
    ('classify', LogisticRegression(max_iter=300, class_weight='balanced', random_state=12))
])

In [20]:
pipe_cluster.fit(X=X_train, y=y_train)
print('Качество модели на обучающей выборке:',\
      {roc_auc_score(pd.get_dummies(y_train), pd.get_dummies(pipe_cluster.predict(X_train)), average='macro', multi_class='ovr')})

print('Качество модели на валидационной выборке:',\
      {roc_auc_score(pd.get_dummies(y_val), pd.get_dummies(pipe_cluster.predict(X_val)), average='macro', multi_class='ovr')})

print('Качество модели на тестовой выборке:',\
      {roc_auc_score(pd.get_dummies(y_test), pd.get_dummies(pipe_cluster.predict(X_test)), average='macro', multi_class='ovr')})

Качество модели на обучающей выборке: {0.9988721280883466}
Качество модели на валидационной выборке: {0.998779041868587}
Качество модели на тестовой выборке: {0.9985367209633171}


In [21]:
# сериализация модели
with open('models/current_cluster_model.pk', 'wb') as file:
    pickle.dump(pipe_cluster, file)

In [22]:
# проверка правильности загрузки модели
with open('models/current_cluster_model.pk','rb') as f:
    loaded_model = pickle.load(f)
    
print(
    'Качество модели на тестовой выборке от законсервированной модели:', 
    {roc_auc_score(pd.get_dummies(y_test), pd.get_dummies(loaded_model.predict(X_test)), average='macro', multi_class='ovr')}
)

Качество модели на тестовой выборке от законсервированной модели: {0.9985367209633171}
